In [1]:
%load_ext autoreload
%autoreload
import os, json
from types import SimpleNamespace
from experiment import run_model
from eval import calculate_stats
import pickle
from datetime import datetime 
import torch
from transformers import BertTokenizer
from sklearn.metrics import classification_report
from util.tools import load_config, configEncoder

In [2]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"

In [3]:
config_folder = "config/"
config = load_config(config_folder)


'{"hyperparams": {"NUM_BERT_LAYERS_FREEZE": 8, "MAX_SEQ_LENGTH": "128", "optimizer": "AdamW", "TRAIN_BATCH_SIZE": 12, "EVAL_BATCH_SIZE": 8, "LEARNING_RATE": 1e-05, "NUM_TRAIN_EPOCHS": 1, "NUM_WARMP_STEPS": 100, "WARMUP_PROPORTION": 0.1, "SCHEDULER_STEP_SIZE": 1, "GRADIENT_ACCUMULATION_STEPS": 1, "PIN_MEMORY": "True", "NUM_WORKERS": 4}, "programsettings": {"DATA_DIR": "data_divided/", "BERT_MODEL": "bert-base-cased", "TASK_NAME": "re", "OUTPUT_DIR": "outputs/re/", "REPORTS_DIR": "reports/re/", "CACHE_DIR": "cache/", "WEIGHTS_NAME": "pytorch_model.bin", "MODEL_NAME": "BioBERT_fc", "DEBUG_PRINT": 0}}'

In [4]:
train_inputs, train_label_ids, train_preds, train_loss, dev_inputs, dev_label_ids, dev_loss, dev_preds = run_model(config, device)

INFO:root:***** Running training *****
INFO:root:  Num examples = 99
INFO:root:  Batch size = 12
INFO:root:  Num steps = 16
Epoch:   0%|                                                                                     | 0/2 [00:00<?, ?it/s]

C:\ProgramData\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
Epoch:  50%|██████████████████████████████████████▌                                      | 1/2 [00:36<00:36, 36.12s/it]


so far epoch accuracy:  0.1111111111111111


Epoch: 100%|█████████████████████████████████████████████████████████████████████████████| 2/2 [01:13<00:00, 36.65s/it]


so far epoch accuracy:  0.1414141414141414
Training complete
Best val Acc: 0.141414


INFO:root:***** Running training *****
INFO:root:  Num examples = 99
INFO:root:  Batch size = 12
INFO:root:  Num steps = 16


In [5]:
train_inputs, train_label_ids, train_preds, train_loss, dev_inputs, dev_label_ids, dev_loss, dev_preds

(array([[  101,   123,   119, ...,     0,     0,     0],
        [  101,   108,   156, ...,     0,     0,     0],
        [  101,  1124,  1145, ...,     0,     0,     0],
        ...,
        [  101,   118,  5351, ...,     0,     0,     0],
        [  101,  1230,  1249, ...,     0,     0,     0],
        [  101,  2508, 13923, ...,   142,   119,   102]], dtype=int64),
 array([3, 0, 0, 7, 1, 4, 6, 1, 0, 3, 6, 5, 2, 6, 3, 1, 2, 5, 5, 6, 3, 0,
        0, 0, 0, 2, 4, 4, 0, 2, 0, 5, 2, 2, 5, 0, 0, 6, 0, 0, 1, 0, 1, 0,
        3, 3, 6, 5, 5, 0, 1, 6, 1, 1, 1, 4, 6, 5, 4, 4, 0, 0, 2, 2, 3, 5,
        1, 2, 2, 2, 6, 4, 4, 3, 6, 6, 0, 3, 0, 6, 5, 5, 6, 6, 0, 0, 1, 2,
        1, 2, 1, 1, 3, 7, 6, 6, 2, 4, 2], dtype=int64),
 array([1, 1, 4, 7, 7, 6, 1, 1, 1, 6, 3, 1, 7, 1, 1, 6, 6, 1, 1, 3, 6, 1,
        1, 3, 7, 6, 3, 6, 5, 7, 1, 6, 6, 1, 1, 7, 3, 1, 6, 7, 4, 4, 3, 7,
        6, 7, 1, 6, 6, 7, 1, 7, 1, 3, 1, 1, 6, 6, 7, 6, 1, 3, 1, 1, 1, 1,
        1, 7, 1, 1, 6, 1, 6, 6, 7, 6, 1, 6, 7, 1, 1, 6, 

In [6]:
print(classification_report(dev_label_ids,dev_preds ))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        18
           1       0.03      0.20      0.06         5
           2       0.00      0.00      0.00        10
           3       0.00      0.00      0.00         7
           4       0.00      0.00      0.00         3
           5       0.00      0.00      0.00         6
           6       0.00      0.00      0.00         5
           7       0.05      0.50      0.08         2
           8       0.50      0.02      0.04        43

    accuracy                           0.03        99
   macro avg       0.06      0.08      0.02        99
weighted avg       0.22      0.03      0.02        99



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [7]:
def save_missed_cases_to_file(file_start_name, dev_preds, dev_label_ids, train_inputs):
    tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)
    missed_cases = []
    for i in range(0,50):
        if dev_label_ids[i] !=  dev_preds[i]:
             missed_cases.append([ dev_preds[i],  dev_label_ids[i] , " ". join (tokenizer.convert_ids_to_tokens(train_inputs[i])) ])

    #Save into a file
    missed_cases_file = config.programsettings.REPORTS_DIR +file_start_name + str(datetime.now()).replace(":", "_").replace(".", "_") + ".pkl"
    with open(missed_cases_file, "wb") as f:
        pickle.dump(missed_cases, f)  
        
save_missed_cases_to_file("BIOBERT_fc_missedcases_" , dev_preds, dev_label_ids, train_inputs)

In [8]:
import numpy as np
np.array(dev_label_ids).shape, np.array(dev_preds).shape, np.array(train_preds).shape, np.array(train_label_ids).shape

((99,), (99,), (99,), (99,))

In [9]:
%autoreload
train_mcc, train_f1_score, train_df_results, train_label_matches_df = calculate_stats(train_label_ids,train_preds )
dev_mcc, dev_f1_score, dev_df_results, dev_label_matches_df = calculate_stats(dev_label_ids,dev_preds )


 label: [3 0 0 7 1 4 6 1 0 3 6 5 2 6 3 1 2 5 5 6 3 0 0 0 0 2 4 4 0 2 0 5 2 2 5 0 0
 6 0 0 1 0 1 0 3 3 6 5 5 0 1 6 1 1 1 4 6 5 4 4 0 0 2 2 3 5 1 2 2 2 6 4 4 3
 6 6 0 3 0 6 5 5 6 6 0 0 1 2 1 2 1 1 3 7 6 6 2 4 2]

 preds: [1 1 4 7 7 6 1 1 1 6 3 1 7 1 1 6 6 1 1 3 6 1 1 3 7 6 3 6 5 7 1 6 6 1 1 7 3
 1 6 7 4 4 3 7 6 7 1 6 6 7 1 7 1 3 1 1 6 6 7 6 1 3 1 1 1 1 1 7 1 1 6 1 6 6
 7 6 1 6 7 1 1 6 7 6 1 5 6 6 6 1 4 1 1 3 1 7 6 7 6]

 label: [8 8 8 4 0 4 8 0 8 8 8 8 8 8 5 8 8 2 8 8 5 0 1 0 8 8 6 6 1 8 8 5 8 6 0 5 3
 6 3 8 2 8 0 0 0 2 0 8 4 8 8 3 8 7 0 1 8 8 0 8 5 8 2 8 0 3 8 8 0 8 8 6 8 2
 3 0 8 8 0 0 2 1 8 0 2 8 2 8 3 3 7 1 8 8 5 0 8 2 2]

 preds: [7 5 1 6 1 6 5 6 6 6 5 1 7 5 6 7 5 6 5 4 7 4 7 6 5 1 7 1 1 1 5 1 7 3 6 7 1
 1 1 3 1 6 7 1 3 1 1 5 7 7 4 1 7 7 6 4 6 7 1 1 7 5 7 6 7 1 6 6 7 6 1 8 4 1
 1 1 1 1 6 7 7 6 6 1 7 5 1 7 1 1 1 6 8 6 3 1 6 6 6]


In [10]:
all_experiment_results = []
all_experiment_results.append([config, train_loss, dev_loss, train_mcc, train_f1_score,dev_mcc,dev_f1_score, 
                               dev_label_ids, dev_preds,train_label_ids,train_preds  ])
all_experiment_results

[[<util.tools.config at 0x2396db5a100>,
  2.2273104985555015,
  2.285797701941596,
  -0.009717224655072746,
  0.06407342657342657,
  -0.0289484367779629,
  0.020370370370370372,
  array([8, 8, 8, 4, 0, 4, 8, 0, 8, 8, 8, 8, 8, 8, 5, 8, 8, 2, 8, 8, 5, 0,
         1, 0, 8, 8, 6, 6, 1, 8, 8, 5, 8, 6, 0, 5, 3, 6, 3, 8, 2, 8, 0, 0,
         0, 2, 0, 8, 4, 8, 8, 3, 8, 7, 0, 1, 8, 8, 0, 8, 5, 8, 2, 8, 0, 3,
         8, 8, 0, 8, 8, 6, 8, 2, 3, 0, 8, 8, 0, 0, 2, 1, 8, 0, 2, 8, 2, 8,
         3, 3, 7, 1, 8, 8, 5, 0, 8, 2, 2], dtype=int64),
  array([7, 5, 1, 6, 1, 6, 5, 6, 6, 6, 5, 1, 7, 5, 6, 7, 5, 6, 5, 4, 7, 4,
         7, 6, 5, 1, 7, 1, 1, 1, 5, 1, 7, 3, 6, 7, 1, 1, 1, 3, 1, 6, 7, 1,
         3, 1, 1, 5, 7, 7, 4, 1, 7, 7, 6, 4, 6, 7, 1, 1, 7, 5, 7, 6, 7, 1,
         6, 6, 7, 6, 1, 8, 4, 1, 1, 1, 1, 1, 6, 7, 7, 6, 6, 1, 7, 5, 1, 7,
         1, 1, 1, 6, 8, 6, 3, 1, 6, 6, 6], dtype=int64),
  array([3, 0, 0, 7, 1, 4, 6, 1, 0, 3, 6, 5, 2, 6, 3, 1, 2, 5, 5, 6, 3, 0,
         0, 0, 0, 2, 4, 4, 0, 2, 

In [11]:
dev_label_matches_df

predicted
labels         matched           
ADE-Drug       False            1
               True             1
Dosage-Drug    False            5
Duration-Drug  False            3
Form-Drug      False            6
Frequency-Drug False            7
Reason-Drug    False           18
Route-Drug     False            4
               True             1
Strength-Drug  False           10
no relation    False           42
               True             1

##  Try with BERT Sequential configuration

### Just change model from BIOR to BERT Sequence

In [12]:
config.programsettings.MODEL_NAME = "BERT_Sequence"
config.programsettings.DEBUG_PRINT = 0

In [ ]:
train_inputs, train_label_ids, train_preds, train_loss, dev_inputs, dev_label_ids, dev_loss, dev_preds = run_model(config, device)

INFO:root:***** Running training *****
INFO:root:  Num examples = 99
INFO:root:  Batch size = 12
INFO:root:  Num steps = 16
INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased.tar.gz from cache at cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c
INFO:pytorch_pretrained_bert.modeling:extracting archive file cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c to temp dir C:\Users\pnars\AppData\Local\Temp\tmp6ubgse3f
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size":

C:\ProgramData\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
Epoch:  50%|██████████████████████████████████████▌                                      | 1/2 [00:46<00:46, 46.21s/it]


so far epoch accuracy:  0.09090909090909091


In [ ]:

train_mcc, train_f1_score, train_df_results, train_label_matches_df = calculate_stats(train_label_ids,train_preds )
dev_mcc, dev_f1_score, dev_df_results, dev_label_matches_df = calculate_stats(dev_label_ids,dev_preds)

In [ ]:
all_experiment_results.append([config, train_loss, dev_loss, train_mcc, train_f1_score,dev_mcc,dev_f1_score, 
                               dev_label_ids, dev_preds,train_label_ids,train_preds  ])
all_experiment_results

In [ ]:
dev_label_matches_df

In [ ]:

all_model_results_pickle_file = config.programsettings.REPORTS_DIR + "multi_model_experiment_results_" + str(datetime.now()).replace(":", "_").replace(".", "_") + ".pkl"
with open(all_model_results_pickle_file, "wb") as f:
    pickle.dump(all_experiment_results, f)  


In [ ]:
print(classification_report(dev_label_ids,dev_preds ))

In [ ]:
# Save missed cases.
save_missed_cases_to_file("BERT_sequential_missedcases_" , dev_preds, dev_label_ids, train_inputs)


# Hyper paramter tuning based on experiments in experiments_batch

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"
from  experiments_batch import run_all_experiments_save
run_all_experiments_save(device)

INFO:root:***** Running training *****
INFO:root:  Num examples = 61014
INFO:root:  Batch size = 24
INFO:root:  Num steps = 25420
Epoch:   0%|                                                                                    | 0/10 [00:00<?, ?it/s]